# Coursework 2 B

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the instructions file located under spec/coursework2.pdf. The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

### Part B
Construct and train networks containing one and two convolutional layers, and max-pooling layers, using
the Balanced EMNIST data, reporting your experimental results. As a default use convolutional kernels of
dimension 5x5 (stride 1) and pooling regions of 2x2 (stride 2, hence non-overlapping). As a default convo-
lutional networks with two convolutional layers, investigate a network with two convolutional+maxpooling
layers with 5 feature maps in the first convolutional layer, and 10 feature maps in the second convolutional
layer.

In [14]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for 5 epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    
    return optimiser, stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import numpy as np
from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.layers import ConvolutionalLayer, MaxPoolingLayer, ReshapeLayer, BatchNormalizationLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.learning_rules import RMSPropLearningRule, AdamLearningRule
from mlp.optimisers import Optimiser
from mlp.penalty import L1Penalty, L2Penalty
from collections import defaultdict
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider
################################################################################
# save and present the data
save_stats= defaultdict()
save_results = defaultdict()
def save_and_present(experiment, stats, parameter):

    np.savetxt(experiment +'.csv', stats, delimiter=',')

    error_valid= stats[1:, keys['error(valid)']]
    error_train= stats[1:, keys['error(train)']]
    acc_valid = stats[1:, keys['acc(valid)']]

    file = open(experiment+'_'+str(parameter)+'.txt','w')

    overfitting = error_valid-error_train
    file.write('Experiment '+experiment+' best acc at Epoch={} by parameter={}\n'.
          format(np.argmax(acc_valid)+1, parameter))
    file.write('error(train)= {}, error(valid)={}, \n error gap = {},  acc(valid)={}\n'.
          format(error_train[np.argmax(acc_valid)],error_valid[np.argmax(acc_valid)],overfitting[np.argmax(acc_valid)], max(acc_valid)))
    file.write('Smallest error gap(after best acc epoch) = {} at Epoch={}'.
          format(min(overfitting[np.argmax(acc_valid):]),np.argmin(overfitting[np.argmax(acc_valid):])+np.argmax(acc_valid)+1))
    print('Experiment '+experiment+' best acc at Epoch={} by parameter={}\n'.
          format(np.argmax(acc_valid)+1, parameter))
    print('error(train)= {}, error(valid)={}, \n error gap = {},  acc(valid)={}\n'.
          format(error_train[np.argmax(acc_valid)],error_valid[np.argmax(acc_valid)],overfitting[np.argmax(acc_valid)], max(acc_valid)))
    print('Smallest error gap(after best acc epoch) = {} at Epoch={}'.
          format(min(overfitting[np.argmax(acc_valid):]),np.argmin(overfitting[np.argmax(acc_valid):])+np.argmax(acc_valid)+1))


In [6]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 10102016 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the EMNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)
test_data = EMNISTDataProvider('test', batch_size=batch_size, rng=rng)


['inputs', 'targets']
['inputs', 'targets']
['inputs', 'targets']


In [7]:
print(train_data.next()[0].shape)
test_data.inputs.shape

(100, 784)


(15800, 784)

### Experiments
1. Filter size: Above we used 7x7; this makes pretty pictures but smaller filters may be more efficient
2. Number of filters: Above we used 32 filters. Do more or fewer do better?
3. Batch normalization: Try adding vanilla batch normalization aafter affine layers. Do your networks train faster?
4. Network architecture: The network above has two layers of trainable parameters. Can you do better with a deeper network? You can implement alternative architectures in the file cs231n/classifiers/convnet.py. Some good architectures to try include:
    1. [conv-relu-pool]xN - conv - relu - [affine]xM - [softmax or SVM]
    2. [conv-relu-pool]XN - [affine]XM - [softmax or SVM]
    3. [conv-relu-conv-relu-pool]xN - [affine]xM - [softmax or SVM]


#### Con_relu_pool_x2_af_re_af_sgd

In [ ]:
rng.seed(seed)

#setup hyperparameters
learning_rate = 1e-3
num_epochs = 1
stats_interval = 1

pad=0
stride=1
# kernel shape and feature maps
num_output_channels1, num_output_channels2, kernel_dim_1, kernel_dim_2 = 5,10,5,5
# Initial input, final output shape
inputs_units, output_dim, hidden_dim = 784, 47, 256
#####################################################################################################
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer is (batch_size, num_output_channels, Con_out_1, Con_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
# Flatten the image for relu
# Reshape to image shape for first Maxpool
# the ouput shape of the first Maxpool 
Max_out_1 = Con_out_1//2
#####################################################################################################
# The input shape of the second conv layer

# the ouput shape of the second convol layer is (batch_size, num_output_channels2, Con_out_2, Con_out_2)
Con_out_2 = (Max_out_1 - kernel_dim_1+2*pad)//stride + 1
# Flatten the image for relu
# Rehape to image shape for second maxpool
num_input_channels3, input_dim_1_4, input_dim_2_4 = num_output_channels2, Con_out_2, Con_out_2
# the ouput shape of the second Maxpool 
Max_out_2 = Con_out_2//2
#####################################################################################################
# then flatten the output
Fc_in = num_output_channels2* Max_out_2* Max_out_2

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),

    ConvolutionalLayer(num_input_channels, num_output_channels1, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    ReshapeLayer((num_output_channels1, Con_out_1, Con_out_1)),
    MaxPoolingLayer(),
    
    ConvolutionalLayer(num_output_channels1, num_output_channels2, Max_out_1, Max_out_1, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    ReshapeLayer((num_output_channels2, Con_out_2, Con_out_2)),
    MaxPoolingLayer(),
    
    ReshapeLayer(), 
    AffineLayer(Fc_in, hidden_dim, weights_init, biases_init),
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = GradientDescentLearningRule(learning_rate=learning_rate,)

experiment = 'Con_relu_pool_x2_af_re_af_sgd'

#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=False)
fig_1.savefig(experiment+ '_learning_rate_{}_error.pdf'.format(learning_rate))
fig_2.savefig(experiment+ '_learning_rate_{}_accuracy.pdf'.format(learning_rate))

save_and_present(experiment, stats, learning_rate)

save_stats[experiment] = stats


result = optimiser.eval_monitors(test_data, 'test')
print('Error:    ' + str(result['errortest']))
print('Accuracy: ' + str(result['acctest']))



### Baseline 1: one convolution layer 
INPUT -> CONV -> RELU -> FC

In [ ]:
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.01
num_epochs = 50
stats_interval = 1

pad=0
stride=1
# First layer kernel shape
num_output_channels, kernel_dim_1, kernel_dim_2 = 5,5,5
# Initial input, final output shape
inputs_units, output_dim = 784, 47
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer + maxpool is (batch_size, num_output_channels, Con_out_1, Con_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
#Max_out_1 = Con_out_1//2
# then reshaped to (batch_size, num_output_channels* Con_out_1* Con_out_1)
Fc_in = num_output_channels* Con_out_1* Con_out_1

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)
model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),
    ConvolutionalLayer(num_input_channels, num_output_channels, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    
    #MaxPoolingLayer(),
    #ReshapeLayer(), 
    #ReluLayer(),
    
    AffineLayer(Fc_in, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = GradientDescentLearningRule(learning_rate=learning_rate,)


experiment = 'Con_re_af'

#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
optimiser, stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
fig_1.savefig(experiment+ '_learning_rate_{}_error.pdf'.format(learning_rate))
fig_2.savefig(experiment+ '_learning_rate_{}_accuracy.pdf'.format(learning_rate))

save_and_present(experiment, stats, learning_rate)

save_stats[experiment] = stats

save_results[experiment] = optimiser.eval_monitors(test_data, 'test')
print('Test error:    ' + str(save_results[experiment]['errortest']))
print('Test accuracy: ' + str(save_results[experiment]['acctest']))


HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))


### Baseline 2:  
INPUT -> CONV -> RELU -> POOL -> FC -> RELU -> FC  
Fc hidden_dim=1000

In [ ]:
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.01
num_epochs = 50
stats_interval = 1

pad=0
stride=1
# First layer kernel shape
num_output_channels, kernel_dim_1, kernel_dim_2 = 5,5,5
# Initial input, final output shape
inputs_units, output_dim, hidden_dim = 784, 47, 100
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer + maxpool is (batch_size, num_output_channels, Con_out_1, Con_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_1 = Con_out_1//2
# then reshaped to (batch_size, num_output_channels* Con_out_1* Con_out_1)
Fc_in = num_output_channels* Max_out_1* Max_out_1

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)
model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),

    ConvolutionalLayer(num_input_channels, num_output_channels, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    ReshapeLayer((num_output_channels, Con_out_1, Con_out_1)),
    MaxPoolingLayer(),
    
    ReshapeLayer(),  
    AffineLayer(Fc_in, hidden_dim, weights_init, biases_init)
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = GradientDescentLearningRule(learning_rate=learning_rate,)


experiment = 'Con_re_pool_sgd'

#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=False)
fig_1.savefig(experiment+ '_learning_rate_{}_error.pdf'.format(learning_rate))
fig_2.savefig(experiment+ '_learning_rate_{}_accuracy.pdf'.format(learning_rate))

save_and_present(experiment, stats, learning_rate)

save_stats[experiment] = stats

save_results[experiment] = optimiser.eval_monitors(test_data, 'test')
print('Test error:    ' + str(save_results[experiment]['errortest']))
print('Test accuracy: ' + str(save_results[experiment]['acctest']))

### One Conv+ Maxpooling layer


In [ ]:
%reload_ext autoreload
%autoreload 2

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.layers import ConvolutionalLayer, MaxPoolingLayer, ReshapeLayer, BatchNormalizationLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.learning_rules import RMSPropLearningRule,AdamLearningRule
from mlp.optimisers import Optimiser

# to ensure reproducibility of results
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.001
num_epochs = 100
stats_interval = 1

pad=0
stride=1
# First layer kernel shape
num_output_channels, kernel_dim_1, kernel_dim_2 = 5,5,5
# Initial input, final output shape
inputs_units, output_dim = 784, 47
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer + maxpool is (batch_size, num_output_channels, Con_out_1, Con_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_1 = Con_out_1//2
# then reshaped to (batch_size, num_output_channels* Con_out_1* Con_out_1)
hidden_dim = num_output_channels* Max_out_1* Max_out_1

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)
model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),
    ConvolutionalLayer(num_input_channels, num_output_channels, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    MaxPoolingLayer(),
    ReshapeLayer(), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate,)

save_and_present('Con_pool_x2')

#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
stats_con_max1, keys, run_time, fig_1_con_max1, ax_1, fig_2_con_max1, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
fig_1_con_max1.savefig('error_con1_learning_rate_{}.pdf'.format(learning_rate))
fig_2_con_max1.savefig('accuracy_con1_learning_rate_{}.pdf'.format(learning_rate))

_  = stats_con_max1
error_valid= _[1:, keys['error(valid)']]
error_train= _[1:, keys['error(train)']]
acc_valid = _[1:, keys['acc(valid)']]
overfitting = error_valid-error_train
print('Best acc at Epoch={} by learning_rate={}'.
      format(np.argmax(acc_valid)+1,learning_rate))
print('error(train)= {}, error(valid)={}, \n                             error gap = {},  acc(valid)={}'.
      format(error_train[np.argmax(acc_valid)],error_valid[np.argmax(acc_valid)],overfitting[np.argmax(acc_valid)], max(acc_valid)))
print('Smallest error gap(after best acc epoch) = {} at Epoch={}'.
      format(min(overfitting[np.argmax(acc_valid):]),np.argmin(overfitting[np.argmax(acc_valid):])+np.argmax(acc_valid)+1))

### 2 Convolve+ MaxPooling layer

In [ ]:
%reload_ext autoreload
%autoreload 2

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.layers import ConvolutionalLayer, MaxPoolingLayer, ReshapeLayer, BatchNormalizationLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.learning_rules import RMSPropLearningRule,AdamLearningRule
from mlp.optimisers import Optimiser

# to ensure reproducibility of results
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.001
num_epochs = 50
stats_interval = 1

pad=0
stride=1
# First layer kernel shape
num_output_channels1, num_output_channels2, kernel_dim_1, kernel_dim_2 = 5,10,5,5
# Initial input, final output shape
inputs_units, output_dim = 784, 47
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer + maxpool is (batch_size, num_output_channels, Max_out_1, Max_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_1 = Con_out_1//2
# the ouput shape of the second convol layer + maxpool is (batch_size, num_output_channels2, Max_out_2, Max_out_2)
Con_out_2 = (Max_out_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_2 = Con_out_2//2
# then reshaped to (batch_size, num_output_channels* Con_out_1* Con_out_1)
hidden_dim = num_output_channels2* Max_out_2* Max_out_2

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),
    ConvolutionalLayer(num_input_channels, num_output_channels1, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    MaxPoolingLayer(),
    ConvolutionalLayer(num_output_channels1, num_output_channels2, Max_out_1, Max_out_1, kernel_dim_1, kernel_dim_2),
    MaxPoolingLayer(),
    ReshapeLayer(), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate,)


experiment = 'Con_pool_x2'
#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
fig_1.savefig('error_'+ experiment +'_learning_rate_{}.pdf'.format(learning_rate))
fig_2.savefig('accuracy_'+ experiment +'_learning_rate_{}.pdf'.format(learning_rate))

save_and_present(experiment, stats)

save_stats[experiment] = stats

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2000), HTML(value='')))

Epoch 1: 1622.2s to complete
    error(train)=5.97e-01, acc(train)=8.07e-01, error(valid)=6.03e-01, acc(valid)=8.03e-01


HBox(children=(IntProgress(value=0, max=2000), HTML(value='')))

Epoch 2: 1597.8s to complete
    error(train)=5.37e-01, acc(train)=8.26e-01, error(valid)=5.49e-01, acc(valid)=8.23e-01


...

HBox(children=(IntProgress(value=0, max=2000), HTML(value='')))

Epoch 43: 1536.3s to complete
    error(train)=3.64e-01, acc(train)=8.71e-01, error(valid)=4.27e-01, acc(valid)=8.57e-01


HBox(children=(IntProgress(value=0, max=2000), HTML(value='')))

Epoch 44: 1538.3s to complete
    error(train)=3.79e-01, acc(train)=8.66e-01, error(valid)=4.37e-01, acc(valid)=8.49e-01


HBox(children=(IntProgress(value=0, max=2000), HTML(value='')))

###  Conv - Relu - MaxPool - Conv - Relu - MaxPool
Adding relu layer between conv and pooling layer
[conv-relu-pool]x2 - relu - [softmax]

In [ ]:
%reload_ext autoreload
%autoreload 2

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.layers import ConvolutionalLayer, MaxPoolingLayer, ReshapeLayer, BatchNormalizationLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.learning_rules import RMSPropLearningRule,AdamLearningRule
from mlp.optimisers import Optimiser

# to ensure reproducibility of results
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.001
num_epochs = 30
stats_interval = 1

pad=0
stride=1
# kernel shape and feature maps
num_output_channels1, num_output_channels2, kernel_dim_1, kernel_dim_2 = 5,10,5,5
# Initial input, final output shape
inputs_units, output_dim = 784, 47
#####################################################################################################
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer is (batch_size, num_output_channels, Con_out_1, Con_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
# Flatten the image for relu
# Reshape to image shape for first Maxpool
# the ouput shape of the first Maxpool 
Max_out_1 = Con_out_1//2
#####################################################################################################
# The input shape of the second conv layer

# the ouput shape of the second convol layer is (batch_size, num_output_channels2, Con_out_2, Con_out_2)
Con_out_2 = (Max_out_1 - kernel_dim_1+2*pad)//stride + 1
# Flatten the image for relu
# Rehape to image shape for second maxpool
num_input_channels3, input_dim_1_4, input_dim_2_4 = num_output_channels2, Con_out_2, Con_out_2
# the ouput shape of the second Maxpool 
Max_out_2 = Con_out_2//2
#####################################################################################################
# then flatten the output
hidden_dim = num_output_channels2* Max_out_2* Max_out_2

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),
    ConvolutionalLayer(num_input_channels, num_output_channels1, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    ReshapeLayer((num_output_channels1, Con_out_1, Con_out_1)),
    MaxPoolingLayer(),
    
    ConvolutionalLayer(num_output_channels1, num_output_channels2, Max_out_1, Max_out_1, kernel_dim_1, kernel_dim_2),
    ReshapeLayer(), 
    ReluLayer(),
    MaxPoolingLayer(),
    ReshapeLayer(), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate,)

_exm = 'Con_relu_max_x2'

stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
fig_1.savefig('error_'+ _exm +'_learning_rate_{}.pdf'.format(learning_rate))
fig_2.savefig('accuracy_'+ _exm +'_learning_rate_{}.pdf'.format(learning_rate))

#_ = stats_con_max2

save_stats[_exm]=stats

np.savetxt(_exm +'.csv', stats, delimiter=',')

error_valid= stats[1:, keys['error(valid)']]
error_train= stats[1:, keys['error(train)']]
acc_valid = stats[1:, keys['acc(valid)']]

overfitting = error_valid-error_train
print('Best acc at Epoch={} by learning_rate={}'.
      format(np.argmax(acc_valid)+1,learning_rate))
print('error(train)= {}, error(valid)={}, \n                             error gap = {},  acc(valid)={}'.
      format(error_train[np.argmax(acc_valid)],error_valid[np.argmax(acc_valid)],overfitting[np.argmax(acc_valid)], max(acc_valid)))
print('Smallest error gap(after best acc epoch) = {} at Epoch={}'.
      format(min(overfitting[np.argmax(acc_valid):]),np.argmin(overfitting[np.argmax(acc_valid):])+np.argmax(acc_valid)+1))

### 2 Converlution+ MaxPooling layer with batch normalisation 
Adding vanilla batch normalization after affine layers.


In [ ]:
%reload_ext autoreload
%autoreload 2

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer, ELULayer, SELULayer
from mlp.layers import ConvolutionalLayer, MaxPoolingLayer, ReshapeLayer, BatchNormalizationLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.learning_rules import RMSPropLearningRule,AdamLearningRule
from mlp.optimisers import Optimiser

# to ensure reproducibility of results
rng.seed(seed)
#train_data.reset()
#valid_data.reset()

#setup hyperparameters
learning_rate = 0.001
num_epochs = 30
stats_interval = 1

pad=0
stride=1
# First layer kernel shape
num_output_channels1, num_output_channels2, kernel_dim_1, kernel_dim_2 = 5,10,5,5
# Initial input, final output shape
inputs_units, output_dim = 784, 47
# Rehape to image shape for first convol
num_input_channels, input_dim_1, input_dim_2 = 1, 28, 28
# the ouput shape of the first convol layer + maxpool is (batch_size, num_output_channels, Max_out_1, Max_out_1)
Con_out_1 =  (input_dim_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_1 = Con_out_1//2
# the ouput shape of the second convol layer + maxpool is (batch_size, num_output_channels2, Max_out_2, Max_out_2)
Con_out_2 = (Max_out_1 - kernel_dim_1+2*pad)//stride + 1 
Max_out_2 = Con_out_2//2
# then reshaped to (batch_size, num_output_channels* Con_out_1* Con_out_1)
hidden_dim = num_output_channels2* Max_out_2* Max_out_2

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model = MultipleLayerModel([
    ReshapeLayer((num_input_channels,input_dim_1,input_dim_2)),
    ConvolutionalLayer(num_input_channels, num_output_channels1, input_dim_1, input_dim_2, kernel_dim_1, kernel_dim_2),
    MaxPoolingLayer(),
    ConvolutionalLayer(num_output_channels1, num_output_channels2, Max_out_1, Max_out_1, kernel_dim_1, kernel_dim_2),
    MaxPoolingLayer(),
    ReshapeLayer(), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate,)

#return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2
stats_con_max2, keys, run_time, fig_1_con_max2, ax_1, fig_2_con_max2, ax_2 = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
fig_1_con_max2.savefig('error_con_max2_learning_rate_{}.pdf'.format(learning_rate))
fig_2_con_max2.savefig('accuracy_con_max2_learning_rate_{}.pdf'.format(learning_rate))

_stats_name = 'stats_con_max2'
_ = stats_con_max2

save_stats[_stats_name] = _
np.savetxt(_stats_name +'.csv', _, delimiter=',')

error_valid= _[1:, keys['error(valid)']]
error_train= _[1:, keys['error(train)']]
acc_valid = _[1:, keys['acc(valid)']]
overfitting = error_valid-error_train
print('Best acc at Epoch={} by learning_rate={}'.
      format(np.argmax(acc_valid)+1,learning_rate))
print('error(train)= {}, error(valid)={}, \n                             error gap = {},  acc(valid)={}'.
      format(error_train[np.argmax(acc_valid)],error_valid[np.argmax(acc_valid)],overfitting[np.argmax(acc_valid)], max(acc_valid)))
print('Smallest error gap(after best acc epoch) = {} at Epoch={}'.
      format(min(overfitting[np.argmax(acc_valid):]),np.argmin(overfitting[np.argmax(acc_valid):])+np.argmax(acc_valid)+1))

### Adam learning rules